In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *
from sloppy.metrics import *
#from ipywidgets import Layout, IntSlider, FloatLogSlider, FloatSlider, interactive, fixed
import inspect

In [3]:
from ipywidgets import *

In [4]:
#def LensCavCAD(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, lens_diam=6.35, lens_thick=4., Rlens=5.0):
#    return LensCav(arm1=arm1, arm2=arm2, base=base, angle=angle, lens_dist=lens_dist, lens_diam=lens_diam, lens_thick=lens_thick, Rlens=Rlens)
def LensCavCADpert(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, Rlens=5.0, c4=6e-4, c6=0., c8=0., eps=np.zeros((12,5))):
    angle = np.deg2rad(angle)
    lens_diam=6.35
    lens_thick=4.
    quart_thick=2.
    quart_dist=1e-6
    
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    pa = np.array([lens_dist/2.+lens_thick/2.+quart_dist, 0, 0])
    pb = np.array([lens_dist/2.+lens_thick/2.+quart_dist+quart_thick,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    pc = np.array([-lens_dist/2.-lens_thick/2.-quart_dist-quart_thick,0,0])
    pd = np.array([-lens_dist/2.-lens_thick/2.-quart_dist,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])

    ps = np.stack([p0,p1,pa,pb,p2,p3,p4,p5,pc,pd,p6,p7], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
        
    #apply poition perturbations
    ps += eps[:,:3]
    ns = [rot_vec(rot_vec(ns[i], ax_x[i], np.deg2rad(eps[i,3])), ax_y[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]    
        
    hi = 12.7
    qi= 7.75
    ng = 1.4537
    coef = np.zeros(9)
    coef[4] = c4
    coef[6] = c6
    coef[8] = c8
    elements = [\
                Glass(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=lens_diam, n2=ng),\
                CurvedGlass(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=lens_diam, R=-Rlens, curv='CC', n1=ng),\
                FreeFormInterface(p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=lens_diam, n2=ng, coef=coef),\
                Glass(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=lens_diam, n1=ng),\
                Mirror(p=ps[4], n=ns[4], ax=ax_x[4], ay=ax_y[4], Rbasis=Rtr[4], diameter=hi),\
                Mirror(p=ps[5], n=ns[5], ax=ax_x[5], ay=ax_y[5], Rbasis=Rtr[5], diameter=hi),\
                Mirror(p=ps[6], n=ns[6], ax=ax_x[6], ay=ax_y[6], Rbasis=Rtr[6], diameter=hi),\
                Mirror(p=ps[7], n=ns[7], ax=ax_x[7], ay=ax_y[7], Rbasis=Rtr[7], diameter=hi),\
                Glass(p=ps[8], n=ns[8], ax=ax_x[8], ay=ax_y[8], Rbasis=Rtr[8], diameter=lens_diam, n2=ng),\
                FreeFormInterface(p=ps[9], n=ns[9], ax=ax_x[9], ay=ax_y[9], Rbasis=Rtr[9], diameter=lens_diam, n1=ng, coef=-coef),\
                CurvedGlass(p=ps[10], n=ns[10], ax=ax_x[10], ay=ax_y[10], Rbasis=Rtr[10], diameter=lens_diam, R=Rlens, curv='CX', n2=ng),\
                Glass(p=ps[11], n=ns[11], ax=ax_x[11], ay=ax_y[11], Rbasis=Rtr[11], diameter=lens_diam, n1=ng)\
                ]
    return elements

cavfct = LensCavCADpert
parname = 'lens_dist'

In [4]:
def LensCavPert(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, Rlens=5.0, eps=np.zeros((8,5))):
    angle = np.deg2rad(angle)
    lens_diam=6.35
    lens_thick=4.
    
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])
    ps = np.stack([p0,p1,p2,p3,p4,p5,p6,p7], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
    
    for i in range(ps.shape[0]):
        ps[i] += eps[i,0]*ax_x[i] + eps[i,1]*ax_y[i] + eps[i,2]*ns[i]
    
    #apply poition perturbations
    #ps += eps[:,:3]
    ns = [rot_vec(rot_vec(ns[i], ax_x[i], np.deg2rad(eps[i,3])), ax_y[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]   
        
    hi = 12.7
    qi=7.75
    ng = 1.4537
     #negative sign of firstt cuved surface for abcd matrix
    elements = [Glass(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=lens_diam, n2=ng),\
                CurvedGlass(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=lens_diam, R=-Rlens, curv='CC', n1=ng),\
                Mirror(p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=hi),\
                Mirror(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=hi),\
                Mirror(p=ps[4], n=ns[4], ax=ax_x[4], ay=ax_y[4], Rbasis=Rtr[4], diameter=hi),\
                Mirror(p=ps[5], n=ns[5], ax=ax_x[5], ay=ax_y[5], Rbasis=Rtr[5], diameter=hi),\
                CurvedGlass(p=ps[6], n=ns[6], ax=ax_x[6], ay=ax_y[6], Rbasis=Rtr[6], diameter=lens_diam, R=Rlens, curv='CX', n2=ng),\
                Glass(p=ps[7], n=ns[7], ax=ax_x[7], ay=ax_y[7], Rbasis=Rtr[7], diameter=lens_diam, n1=ng)]
    #apply position offsets in element coordinates
    #for i, e in enumerate(elements):
    #    e.p += (e.Rot.T)@eps[i,:3]
    return elements

cavfct = LensCavPert
parname = 'lens_dist'

In [4]:
def NoLensCavPert(arm1=48., arm2=30., base=19., angle=180.-129., lens_dist=22.2, Rlens=5.0, eps=np.zeros((4,5))):
    angle = np.deg2rad(angle)
    lens_diam=6.35
    lens_thick=4.
    

    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    ps = np.stack([p2,p3,p4,p5], axis=0)
    
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    Rtr = geom['R']
    ax_x = geom['xin']
    ax_y = np.cross(ns, ax_x)
    
    for i in range(ps.shape[0]):
        ps[i] += eps[i,0]*ax_x[i] + eps[i,1]*ax_y[i] + eps[i,2]*ns[i]
    
    #apply poition perturbations
    #ps += eps[:,:3]
    ns = [rot_vec(rot_vec(ns[i], ax_x[i], np.deg2rad(eps[i,3])), ax_y[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]   
        
    hi = 12.7
    qi=7.75
    ng = 1.4537
     #negative sign of firstt cuved surface for abcd matrix
    elements = [Mirror(p=ps[0], n=ns[0], ax=ax_x[0], ay=ax_y[0], Rbasis=Rtr[0], diameter=hi),\
                Mirror(p=ps[1], n=ns[1], ax=ax_x[1], ay=ax_y[1], Rbasis=Rtr[1], diameter=hi),\
                CurvedMirror(R=1e3, p=ps[2], n=ns[2], ax=ax_x[2], ay=ax_y[2], Rbasis=Rtr[2], diameter=hi),\
                Mirror(p=ps[3], n=ns[3], ax=ax_x[3], ay=ax_y[3], Rbasis=Rtr[3], diameter=hi)]
    #apply position offsets in element coordinates
    #for i, e in enumerate(elements):
    #    e.p += (e.Rot.T)@eps[i,:3]
    return elements

cavfct = NoLensCavPert
parname = 'lens_dist'

In [5]:
res = degeneracy_length(LensCavPert, parname, 5e-2, s=1, degmodenum=2)
Ldeg=res.x
print(Ldeg)

21.77801052565133


In [6]:
waists_vs_param(cavfct, parname, 5e-2, s=1, degmodenum=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

21.780501672240803

In [7]:
try:
    plot.close()
except:
    pass
elements_ref = cavfct(**{parname: Ldeg})
#elements = cavfct(**{parname: Ldeg, 'eps': eps})

plot = k3d.plot(camera_auto_fit=True, antialias=True, camera_fov=20.)

for i, el in enumerate(elements_ref):
    plot += el.plot(opacity=0.4)
plot

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], ca…

In [8]:
elements = cavfct(**{parname: Ldeg})
sys = RaySystem(elements)
system = sys.abcd
print(system.Ltot)
system.waist_at(0)

131.24995092745596


array([0.01720324, 0.01720324])

In [9]:
x0 = np.linspace(0, system.Ltot, 500)
ws0 = system.compute_waists(x0)

plt.figure()
plt.plot(x0, ws0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
ng = 1.4537 #1.460 #1.4537
elements_mm = [Prop(system.distlist[1]), ABCD(elements[0].m), Prop(system.distlist[2]-system.distlist[1], n=ng), ABCD(elements[1].m), Prop(system.distlist[3]-system.distlist[2], n=1),
               ABCD(Glass(n2=ng).m), Prop(5.0, n=ng), ABCD(Glass(n1=ng).m),\
               Prop(500.0),] #

#  ABCD(CurvedGlass(R=69.0, curv='CC', n2=ng).m),Prop(3.8, n=ng),ABCD(Glass(n1=ng).m),\
#                 Prop(205.0), ABCD(CurvedGlass(R=46.0, curv='CC', n2=ng).m),Prop(2.9, n=ng),ABCD(Glass(n1=ng).m),\
#                 Prop(550.0), ABCD(CurvedGlass(R=3.1, curv='CC', n2=ng).m),Prop(2.9, n=ng),ABCD(Glass(n1=ng).m),Prop(3.0)

abcd_mm = ABCDSystem(elements_mm)

In [11]:
x = np.linspace(0, abcd_mm.Ltot, 2000)
ws = abcd_mm.compute_waists(x, qin=system.q_at(0))
#ws2 = abcd_mm_2.compute_waists(x, qin=q_1)
plt.figure()
plt.plot(x0, ws0, ls=':')
ls = plt.plot(x, ws)
plt.axvline(system.distlist[3]+53, color='grey')
#plt.plot(x, ws2, ls=':')
#plt.axhline(0.22*6.2, color='grey')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
ws.shape

(2000, 2)

In [12]:
def interaction_factory(sysfct, pardct, qin=None, Npts=500, wl=780e-6):
    sliders = {n: widgets.FloatSlider(value=v, min=a, max=b) for n, (v,a,b) in pardct.items()}
    
    kwargs = {n: v for n, (v,a,b) in pardct.items()}
    SysObj = ABCDSystem(sysfct(**kwargs), wl=wl)
    
    lrt = SysObj.Ltot
    x = np.linspace(0, lrt, Npts)
    ws = np.zeros_like(x)
    fig, ax = plt.subplots()
    lines = ax.plot(x, ws)
    
    cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    vlinesx = [ax.axvline(d, ls=":", color='grey') for d in SysObj.distlist[1:-1]]
    
    ax.set_xlabel(r"$L$ [mm]")
    ax.set_ylabel(r"Waist $\omega$ [um]")

    def update(**kwargs):
        print(kwargs)
        try:
            SysObj = ABCDSystem(sysfct(**kwargs), wl=wl)
        except AssertionError:
            for line in lines:
                line.set_data(np.array([0,1]),np.array([0,0]))
        else:
            x = np.linspace(0, SysObj.Ltot, Npts)
            y = SysObj.compute_waists(x, qin=qin)
            for i, l in enumerate(lines):
                l.set_data(x,y[:,i])
                
            for i, d in enumerate(SysObj.distlist[1:-1]):
                vlinesx[i].set_data(([d, d], [0, 1]))
            
            ax.relim()
            ax.autoscale_view()
            fig.canvas.draw_idle()

    return fig, ax, interact(update, **sliders)

In [13]:
class Lens(ABCD):
    def __init__(self, f):
        # Returns the ABCD array for a thin lens with focal length f
        self.f = f
        m = np.identity(4, dtype=np.float64)
        m[2,0] = -1.0/f
        m[3,1] = -1.0/f
        self.m = m

In [17]:
system.distlist[3]-system.distlist[2]

11.110994737174336

In [14]:
def SysMM(f, d1, fa, da):
    ng = 1.4537 #1.460 #1.4537
    sys = [Prop(system.distlist[1]), ABCD(elements[0].m), Prop(system.distlist[2]-system.distlist[1], n=ng), ABCD(elements[1].m), Prop(system.distlist[3]-system.distlist[2], n=1),
               ABCD(Glass(n2=ng).m), Prop(5.0, n=ng), ABCD(Glass(n1=ng).m),\
               Prop(53+d1),Lens(f), Prop(300-da),Lens(fa),Prop(da)] # 
    return sys

ParMM = {"f": (400, 10, 500), "d1": (380, 220, 500), "fa": (11, 6, 15), "da": (11, 6, 15)}
_,_,inter = interaction_factory(SysMM, ParMM, qin=system.q_at(0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=400.0, description='f', max=500.0, min=10.0), FloatSlider(value=380.0,…

In [14]:
wfib = 5.3e-3/2
wl = 780e-6
qfib = 1j*np.pi*wfib**2/wl

In [15]:
def SysAsph(fa, da, ltot=100):
    return [Prop(da),Lens(fa),Prop(ltot-da)]

In [17]:
sys_asph = ABCDSystem(SysAsph(11.0, 12.0))
x = np.linspace(0, sys_asph.Ltot, 1000)
ws = sys_asph.compute_waists(x, qin=qfib)

plt.figure()
plt.plot(x, ws, ls=':')

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [82]:
def SysMM1560(f, d1, fa, da):
    ng = 1.4439
    sys = [Prop(system.distlist[1]), ABCD(elements[0].m), Prop(system.distlist[2]-system.distlist[1], n=ng), ABCD(elements[1].m), Prop(system.distlist[3]-system.distlist[2], n=1),
               ABCD(Glass(n2=ng).m), Prop(5.0, n=ng), ABCD(Glass(n1=ng).m),\
               Prop(53+d1),Lens(f), Prop(300-da),Lens(fa),Prop(da)] # 
    return sys
ParMM = {"f": (250, 10, 500), "d1": (225, 200, 500), "fa": (11, 6, 15), "da": (11, 6, 15)}
_, ax, inter = interaction_factory(SysMM1560, ParMM, qin=system.q_at(0),wl=1560e-6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=250.0, description='f', max=500.0, min=10.0), FloatSlider(value=225.0,…